# EV Chargers

In [80]:
# !pip install psycopg2 - if needed

In [81]:
# import dependencies
import psycopg2 as pg
import pandas as pd
from sqlalchemy import create_engine
from bs4 import BeautifulSoup
import matplotlib.pyplot as plt
import requests
import os
import numpy as np
import pandas as pd
import json
import ast
from pprint import pprint

<h3> Update the pw variable </h3>

In [82]:
# enter your postgres password
pw = "postgres"
# make the evData in your postgres

### CSV Imports

In [83]:
#import the GDP csv
csv_file = "data/EV_Vehicle_Registration_State.csv"
ev_df = pd.read_csv(csv_file, encoding ="ISO-8859-1")
ev_df.head()

,State,Registration_Count
0,Alabama,900
1,Alaska,370
2,Arizona,"9,820"
3,Arkansas,270
4,California,"179,600"


In [84]:
#preview the count
ev_df.count()

State                  52
Registration_Count     52
dtype: int64

In [85]:
#get the name of the columns
ev_df.columns

Index(['State', 'Registration_Count '], dtype='object')

In [86]:
#import the countries csv

ccsv_file = "data/alt_fuel.csv"
fuel_df = pd.read_csv(ccsv_file, encoding ="ISO-8859-1")
fuel_df.head()

,Station_Name,Street_Address,City,State,ZIP,Latitude,Longitude,Country
0,City of Sacramento - Capitol Parking Garage,1015 L St,Sacramento,CA,95814,38.578380,-121.492600,US
1,Los Angeles Convention Center,1201 S Figueroa St,Los Angeles,CA,90015,34.040570,-118.268762,US
2,LADWP - John Ferraro Building,111 N Hope St,Los Angeles,CA,90012,34.057922,-118.249800,US
3,Cherokee & Hollywood Parking Garage,1718 N Cherokee Ave,Los Angeles,CA,90028,34.102404,-118.334758,US
4,CITYOFSANTAROSA,100 Santa Rosa Ave,Santa Rosa,CA,95404,38.437875,-122.710897,US


In [87]:
#preview the count

fuel_df.count()

Station_Name      23176
Street_Address    23174
City              23175
State             23175
ZIP               23175
Latitude          23176
Longitude         23176
Country           23176
dtype: int64

In [88]:
#get the name of the columns

fuel_df.columns

Index(['Station_Name', 'Street_Address', 'City', 'State', 'ZIP', 'Latitude',
       'Longitude', 'Country'],
      dtype='object')

### Creating New Data With Select Columns

In [89]:
#ev_df

new_ev_df = ev_df[['State', 'Registration_Count ']].copy()
new_ev_df.head()

,State,Registration_Count
0,Alabama,900
1,Alaska,370
2,Arizona,"9,820"
3,Arkansas,270
4,California,"179,600"


In [90]:
#fuel_df

new_fuel_df = fuel_df[['Station_Name', 'Street_Address', 'City', 'State', 'ZIP', 'Latitude',
       'Longitude', 'Country']].copy()
new_fuel_df.head()

,Station_Name,Street_Address,City,State,ZIP,Latitude,Longitude,Country
0,City of Sacramento - Capitol Parking Garage,1015 L St,Sacramento,CA,95814,38.578380,-121.492600,US
1,Los Angeles Convention Center,1201 S Figueroa St,Los Angeles,CA,90015,34.040570,-118.268762,US
2,LADWP - John Ferraro Building,111 N Hope St,Los Angeles,CA,90012,34.057922,-118.249800,US
3,Cherokee & Hollywood Parking Garage,1718 N Cherokee Ave,Los Angeles,CA,90028,34.102404,-118.334758,US
4,CITYOFSANTAROSA,100 Santa Rosa Ave,Santa Rosa,CA,95404,38.437875,-122.710897,US


### Store The Data Into a DataFrame

In [91]:
#store the data into a evData
rds_connection_string = f"postgres:{pw}@localhost:5432/evData"
engine = create_engine(f'postgresql://{rds_connection_string}')

In [92]:
#check for tables
engine.table_names()

['ev', 'fuel']

### Load csv converted DataFrame into database

In [93]:
new_ev_df.to_sql(name='ev', con=engine, if_exists='append', index=False)

In [94]:
new_fuel_df.to_sql(name='fuel', con=engine, if_exists='append', index=False)

In [95]:
#confirm the ev_data has been added to the table
pd.read_sql_query('select * from ev', con=engine).head()

,State,Registration_Count
0,Alabama,900
1,Alaska,370
2,Arizona,"9,820"
3,Arkansas,270
4,California,"179,600"


In [96]:
#confirm the fuel_data has been added to the table
pd.read_sql_query('select * from fuel', con=engine).head()

,Station_Name,Street_Address,City,State,ZIP,Latitude,Longitude,Country
0,City of Sacramento - Capitol Parking Garage,1015 L St,Sacramento,CA,95814,38.578380,-121.492600,US
1,Los Angeles Convention Center,1201 S Figueroa St,Los Angeles,CA,90015,34.040570,-118.268762,US
2,LADWP - John Ferraro Building,111 N Hope St,Los Angeles,CA,90012,34.057922,-118.249800,US
3,Cherokee & Hollywood Parking Garage,1718 N Cherokee Ave,Los Angeles,CA,90028,34.102404,-118.334758,US
4,CITYOFSANTAROSA,100 Santa Rosa Ave,Santa Rosa,CA,95404,38.437875,-122.710897,US
